# Step 7
You should now have deployed the Analysis Lambda service. We can use the code in this notebook to test the service. Firstly run the code below to view the contents of your data bucket.

In [7]:
// load the AWS SDK
AWS = require('aws-sdk')
support = require('../support')

// create an S3 object to call
s3 = new AWS.S3()

// configure parameters
params = {
    Bucket: process.env.MY_BUCKET_NAME,
//    Prefix: 'rekognition'
}

s3.listObjects(params, function(err, data) {
  support.printS3Objects(data.Contents)
})
$$.clear()


cckkaopdv.ai-as-a-service.s3-website-eu-west-1.amazonaws.com/bcb0cd70-03f0-11eb-9350-b37aa6c39450 (66904)
cckkaopdv.ai-as-a-service.s3-website-eu-west-1.amazonaws.com/bcb56150-03f0-11eb-9350-b37aa6c39450 (27156)
cckkaopdv.ai-as-a-service.s3-website-eu-west-1.amazonaws.com/bcb5d680-03f0-11eb-9350-b37aa6c39450 (85512)
cckkaopdv.ai-as-a-service.s3-website-eu-west-1.amazonaws.com/bcb78430-03f0-11eb-9350-b37aa6c39450 (21958)
cckkaopdv.ai-as-a-service.s3-website-eu-west-1.amazonaws.com/bcb86e90-03f0-11eb-9350-b37aa6c39450 (128672)
cckkaopdv.ai-as-a-service.s3-website-eu-west-1.amazonaws.com/bcbb2db0-03f0-11eb-9350-b37aa6c39450 (304374)
cckkaopdv.ai-as-a-service.s3-website-eu-west-1.amazonaws.com/bcbe8910-03f0-11eb-9350-b37aa6c39450 (574680)
cckkaopdv.ai-as-a-service.s3-website-eu-west-1.amazonaws.com/bcc14830-03f0-11eb-9350-b37aa6c39450 (1246129)
cckkaopdv.ai-as-a-service.s3-website-eu-west-1.amazonaws.com/status.json (1055)
images/2007_iconic_acting_lewis_Cotillard_Swinton_Bardem.jpg (1492

## Post a Message
We will again post a message to our Crawler queue, only this time because the Analysis Service is deployed, it will pick up on a second message that is posted from the Crawler Service to the Analysis queue.

Go ahead and run the code below.

In [6]:
// load the AWS SDK
AWS = require('aws-sdk')
support = require('../support')

// create an SQS object to call
sqs = new AWS.SQS()

// construct Queue URL
queueUrl = `https://sqs.${process.env.AWS_REGION}.amazonaws.com/${process.env.AWS_ACCOUNT_ID}/${process.env.MY_CRAWLER_QUEUE}`

// construct message parameters
params = {
  MessageBody: JSON.stringify({action: 'download', msg: {url: 'http://ai-as-a-service.s3-website-eu-west-1.amazonaws.com'}}),
  QueueUrl: queueUrl
}

// send the message
sqs.sendMessage(params, (err, data) => {
    if (err) {
        console.log(`QUEUE ERROR: ${err}`)
    }
    else {
      console.log(`message id: ${data.MessageId}`)        
    }
})
$$.clear()

message id: 660d5d91-efd4-48a7-84a9-dec59b2f6d13


# 3. Check the Results
If you rerun the code in 1. at the top of this notebook, you should see some additoinal files have been downloaded if our crawler executed successfully. You can confirm this if you look in the AWS S3 console.

Congratulations! You have now deployed and triggered the crawler Lambda functions through SQS and collected some files for analysis.

In the next step we will pass these to the analysis function.